# Spaceship Titanic Competition

*Created by: Taylor Daugherty*

*Created on: 5/23/2023    Last updated: 6/13/2023 - Clean up notebook*

This notebook is used to practice using cross validation.


**Input File:** train.csv and test.csv from the spaceship-titanic competition

- train.csv contains a training set of data with features and true values

- test.csv contains a testing set of data with only features. This is what will be used to make predictions and submit to the competition


**Purpose of notebook:** Gain more practice with cross validation and make submissions to the spaceship titanic competition

## Table of Contents

1. Import packages

2. Useful functions

3. Initial Model

4. Improvement 1: Remove 'Converted' group

5. Improvement 2: Impute NaN values

    a. Constant
    
    b. Mean
    
    c. Median

## Import packages

Import the packages necessary to run the notebook

1. numpy: used for linear algebra

2. pandas: used for dataframe creation and manipulation

4. `SimpleImputer`: Used to fill NaN values

5. `LogisticRegression`: a classification model using logistic regression

6. `KNeighborsClassifier`: a classification model using KNN

7. `SVC`: a classification model using SVM

8. `KFold` and `cross_val_score`: used for cross validation

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedKFold, cross_val_score

In [2]:
train_filepath = '/kaggle/input/spaceship-titanic/train.csv'
test_filepath = '/kaggle/input/spaceship-titanic/test.csv'

### List of models

These are the models that will be used during cross-validation. 

**Models:**

- Logistic Regression

- KNN

- SVM

In [3]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', SVC()))

### apply_imputer()

This function is used in the Improvement 2 section for all three imputation methods. Here, the imputer is applied to the dataframe and the columns are returned to their correct names

**Input:** A simple imputer to apply, 
           a dataframe to impute
           
**Output:** The imputed dataframe

In [4]:
def apply_imputer(si, df):
    df_imputed = pd.DataFrame(si.fit_transform(df))
    df_imputed.columns = df.columns
    return df_imputed

### perform_cross_validation()

This function goes through the steps to perform K-fold cross validation using the list of models described above.

**Input:** A dataframe containing the features use to build the model, a Series of the true values associated with the feature list

**Output:** Printed result for the mean and standard deviation of each model

In [5]:
def perform_cross_validation(X_train, y_train):
    results = dict()

    for name, model in models:
        kfold = KFold(n_splits=10)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
        results[name] = (cv_results.mean(), cv_results.std())

    print('Model\t\tCV Mean\t\tCV std')
    print(results)

In [6]:
def expand_cabin(df):
    df[['Deck','Num','Side']] = df['Cabin'].astype(str).str.split('/', expand=True)
    df = df.drop('Cabin', axis=1)
    df = df.drop('Num', axis=1)
    return df

In [7]:
def fill_na(df, features, fill_vals):
    df_filled = df
    
    for i in range(len(features)):
        feature = features[i]
        fill_val = fill_vals[i]
        
        df_filled[feature] = df_filled[feature].fillna(fill_val)
        
    return df_filled

In [8]:
def map_cat(df, features, dictionaries):
    df_mapped = df
    
    for i in range(len(features)):
        feature = features[i]
        dictionary = dictionaries[i]
        
        df_mapped[feature] = df[feature].map(dictionary)
        
    return df_mapped

## Initial Model

In [9]:
df_train = pd.read_csv(train_filepath, index_col='PassengerId')
df_test = pd.read_csv(test_filepath, index_col='PassengerId')

In [10]:
df_train.head(10)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True


In [11]:
df_train = df_train.drop('Name', axis=1)
df_test = df_test.drop('Name', axis=1)

In [12]:
df_train = expand_cabin(df_train)
df_test = expand_cabin(df_test)

In [13]:
X = df_train.drop('Transported', axis=1)

y = df_train['Transported']

In [14]:
rs_mean = X['RoomService'].mean()
fc_mean = X['FoodCourt'].mean()
sm_mean = X['ShoppingMall'].mean()
spa_mean = X['Spa'].mean()
vr_mean = X['VRDeck'].mean()
age_mean = X['Age'].mean()

rs_test_mean = df_test['RoomService'].mean()
fc_test_mean = df_test['FoodCourt'].mean()
sm_test_mean = df_test['ShoppingMall'].mean()
spa_test_mean = df_test['Spa'].mean()
vr_test_mean = df_test['VRDeck'].mean()
age_test_mean = df_test['Age'].mean()

In [15]:
fill_features = ['HomePlanet', 'Destination', 'Side', 'CryoSleep', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Age']
fill_vals = ['Earth', 'TRAPPIST-1e', 'S', False, False]

fill_vals_train = fill_vals + [rs_mean, fc_mean, sm_mean, spa_mean, vr_mean, age_mean]
fill_vals_test = fill_vals + [rs_test_mean, fc_test_mean, sm_test_mean, spa_test_mean, vr_test_mean, age_test_mean]

In [16]:
X_imputed = fill_na(X, fill_features, fill_vals_train)
X_test_imputed = fill_na(df_test, fill_features, fill_vals_test)
X_imputed.head(10)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Side
PassengerId,,,,,,,,,,,,
0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.000000,B,P
0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.000000,F,S
0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.000000,A,S
0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.000000,A,S
0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.000000,F,S
0005_01,Earth,False,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.000000,F,P
0006_01,Earth,False,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.000000,F,S
0006_02,Earth,True,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,304.854791,G,S
0007_01,Earth,False,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.000000,F,S


In [17]:
map_features = ['HomePlanet','Destination','Deck','Side']

map_dictionaries = [{'Earth':2, 'Europa':1, 'Mars':0}, 
                    {'TRAPPIST-1e':2, '55 Cancri e':1, 'PSO J318.5-22':0}, 
                    {'T':0, 'A':1, 'D':2, 'C':3, 'B':4, 'E':5, 'G':6, 'F':7}, 
                    {'P':0, 'S':1}]

In [18]:
X_mapped = map_cat(X_imputed, map_features, map_dictionaries)
X_test_mapped = map_cat(X_test_imputed, map_features, map_dictionaries)

X_mapped.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Side
PassengerId,,,,,,,,,,,,
0001_01,1,False,2,39.0,False,0.0,0.0,0.0,0.0,0.0,4.0,0
0002_01,2,False,2,24.0,False,109.0,9.0,25.0,549.0,44.0,7.0,1
0003_01,1,False,2,58.0,True,43.0,3576.0,0.0,6715.0,49.0,1.0,1
0003_02,1,False,2,33.0,False,0.0,1283.0,371.0,3329.0,193.0,1.0,1
0004_01,2,False,2,16.0,False,303.0,70.0,151.0,565.0,2.0,7.0,1
